In [1]:
import pandas as pd
import numpy as np

In [32]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from re import sub
from nltk import download
download('wordnet')
download('omw-1.4')
download('punkt')
download('stopwords')

def remove_num(text):
    text = sub(r'\d+', '', text)
    text = sub(r'\s+', ' ',text)
    return text

def remove_punct(text):
    text = sub(r"[!#$%&'()*+,-./:;<=>?@[^_`{|}~]+", ' ',text)
    text = sub(r'\s+', ' ',text)
    return text

def extract_keywords(text):
    tokens = word_tokenize(text)
    keywords = []
    for word in tokens:
        word = word.lower()
        if word not in stopwords.words('portuguese') or word.lower() not in STOP_WORDS:
            keywords.append(word)
    return ' '.join(keywords)

# def get_synonyms(text):
#     tokens = word_tokenize(text)
#     synonyms = []
#     for word in tokens:
#         for syn in wordnet.synsets(word, lang="por"):
#             for lemma in syn.lemmas(lang="por"):
#                 synonyms.append(lemma.name())
#     return synonyms

def preprocess_lemma(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmas = []
    for token in tokens:
        lemmas.append(lemmatizer.lemmatize(token))
    lemmas = ' '.join(lemmas)
    return lemmas

def remove_accent(text):
    text = sub('[áàãâä]', 'a', sub('[éèêë]', 'e', sub('[íìîï]', 'i', sub('[óòõôö]', 'o', sub('[úùûü]', 'u', text)))))
    text = sub(r'\s+', ' ',text)
    return text

def preprocess_stem(text):
    stemmer = SnowballStemmer("portuguese")
    tokens = word_tokenize(text)
    stems = []
    for token in tokens:
        stems.append(stemmer.stem(token))
    stems = ' '.join(stems)
    return stems



def preprocess(text):
    text = remove_punct(text)
    text = remove_num(text)
    # text = extract_keywords(text)
    # text = preprocess_lemma(text)
    text = remove_accent(text)
    return text

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from random import choice

def load_txt(path):
    with open(path,'r', encoding='utf-8') as file:
        linhas = file.readlines()
        lines = []
        for linha in linhas:
            linha = linha.strip()
            if linha != '':
                lines.append(linha)
    return lines

def sintetizar_dados(lista):
    def substituir_sinonimos(frase):
        tokens = nltk.word_tokenize(frase)
        sinonimos = []
        for token in tokens:
            try:
                sinonimo = choice(wordnet.synsets(token)).lemmas()[0].name()
                sinonimos.append(sinonimo if sinonimo != token else token)
            except IndexError:
                sinonimos.append(token)
        return ' '.join(sinonimos)
    def rearranjar_estrutura(frase):
        tokens = nltk.word_tokenize(frase)
        rearranjada = ' '.join(tokens[::-1])
        return rearranjada
    sintetizados = []
    for exemplo in lista:
        exemplo_sinonimos = substituir_sinonimos(exemplo)
        exemplo_rearranjado = rearranjar_estrutura(exemplo)
        sintetizados.extend([exemplo, exemplo_sinonimos, exemplo_rearranjado])
    lista2 = []
    for i,j in zip(lista,sintetizados):
        if i not in lista2:
            lista2.append(i)
        if j not in lista2:
            lista2.append(j)
        i2 = extract_keywords(i)
        j2 = extract_keywords(j)
        if i2 not in lista2:
            lista2.append(i2)
        if j2 not in lista2:
            lista2.append(j2)
    lista3 = lista2
    for x in lista3:
        x = preprocess_lemma(x)
        if x not in lista2:
            lista2.append(x)
    return lista2

def salvar_lista_em_arquivo(lista, nome_arquivo):
    with open(nome_arquivo, 'a+', encoding="utf-8") as arquivo:
        for exemplo in lista:
            arquivo.write(exemplo + '\n')

def eliminar_linhas_repetidas(nome_arquivo):
    linhas_unicas = set()
    with open(nome_arquivo, 'r',encoding="utf-8") as arquivo:
        for linha in arquivo:
            linha = remove_accent(remove_punct(linha)).strip().lower()
            if linha not in linhas_unicas:
                linhas_unicas.add(linha)
    with open(nome_arquivo, 'w',encoding="utf-8") as arquivo:
        for linha in linhas_unicas:
            arquivo.write(linha + '\n')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\herik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\herik\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\herik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\herik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\herik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
def load_txt(path):
    with open(path,'r', encoding='utf-8') as file:
        linhas = file.readlines()
        linhas = sintetizar_dados(linhas)
        print(len(linhas))
        lines = []
        for linha in linhas:
            linha = linha.strip()
            if linha != '':
                lines.append(linha)
    return lines

def create_df(column, lines):
    df_created = pd.DataFrame({column: lines})
    return df_created

def concat_df(list_df):
    df_concated = pd.concat(list_df, axis=1)
    return df_concated

In [34]:
import os

diretorio = r'..\database\problems\problemas txt'

list_df = []
for nome_arquivo in os.listdir(diretorio):
    caminho_completo = os.path.join(diretorio, nome_arquivo)
    column = nome_arquivo[:-4]
    lines = load_txt(caminho_completo)
    df = create_df(column,lines)
    list_df.append(df)
    
new_df = concat_df(list_df)


1149
1084
418
841
709
752
841
888
602
653
842
703
935
829
354
549
398
721
813
708
1106
1078
851
751
879
879
801
920
839
836
598
756
715
581
814
604
498
684
814
1000
695
589
466
621
821
609
667
902
1123
1038
895
1223
1134
1093
819
795
1078
1036
670
666
1006
856
650
620
374
674
896
667
585
789
760
719
795
661
920
624
865
867
614
848
733
679
706
889
668
629
897
761
823
431
671
449
556
532
1356
861
623
735
838
543
837
695
739
1056
862
1048
805
449
667
848
841
753
755
330
404
567


In [35]:
min_size = min([len(new_df[column].dropna()) for column in new_df.columns])
print(min_size)

328


In [6]:
new_df.head()

,arquivo1,arquivo10,arquivo100,arquivo101,arquivo102,arquivo103,arquivo104,arquivo105,arquivo106,arquivo107,...,arquivo90,arquivo91,arquivo92,arquivo93,arquivo94,arquivo95,arquivo96,arquivo97,arquivo98,arquivo99
0,A bomba está superaquecendo.,Consumo de energia elevado na bomba,Ruído agudo e constante vindo do motor elétrico,Ruído de clique regular no motor durante a par...,Barulho de batidas rápidas no motor elétrico,Problema de escova chatter em motores de corre...,Motor elétrico superaquecido,Comutador superaquecido em motores elétricos,Contatos elétricos com soldagem,Contatos elétricos desgastam rapidamente,...,Ruído excessivo durante o funcionamento do mot...,Rolamentos de bucha aquecendo além do normal,Rolamento superaquecido no motor elétrico,Problema de sobrecarga no controlador de motor...,Faíscas nas escovas de motores elétricos,Comutador escurecido em motores elétricos,Faíscas intermitentes nas escovas de motores e...,Poeira fina acumulada no acoplamento com amort...,Fumaça saindo do motor elétrico,Motor elétrico não liga
1,A bomba apresenta temperatura elevada.,Problema de alto consumo de energia na bomba,Zumbido intenso durante a operação do motor,Clique constante no motor elétrico,Vibração intensa e batidas rápidas no motor,Chatter excessivo nas escovas do motor elétrico,Motor esquentando demais,Problema de superaquecimento no comutador de m...,Problema de soldagem nos contatos,Vida útil reduzida dos contatos elétricos,...,Operação barulhenta em motores elétricos,Problema de superaquecimento nos rolamentos de...,Aquecimento excessivo do rolamento antifricção,Controlador apresentando sobrecarga excessiva ...,Flashes excessivos nas escovas de motores,Problema de escurecimento do comutador,Problema de faíscas ocasionais nas escovas de ...,Problema de sujeira sob o acoplamento com pino...,Bigodes de solda na armadura do motor,Falha na partida do motor
2,A bomba está esquentando demais.,Bomba consumindo mais energia do que o normal,Barulho irritante de zumbido no motor principal,Barulho de clique em intervalos regulares no m...,Ruído de batidas rápidas em motores elétricos,Escovas vibrando de forma irregular no motor,Superaquecimento excessivo do motor,Comutador aquecendo demais em motores elétricos,Soldagem indesejada dos contatos elétricos,Problema de curta duração dos contatos elétricos,...,Motor elétrico produzindo ruídos estranhos,Rolamentos de bucha operando em alta temperatura,Problema de temperatura elevada no rolamento d...,Sobrecarga comprometendo o desempenho do contr...,Problema de faíscas em motores elétricos,Comutador escuro em motores,Ocorrência de faíscas intermitentes em escovas...,Acoplamento com amortecedores ou pinos de borr...,Solda derretida formando filamentos no motor,Problemas de partida no motor elétrico
3,O motor da bomba está superaquecido.,Consumo excessivo de energia no funcionamento ...,Excesso de ruído vibrante no motor elétrico,Ruído de clique rítmico no motor principal,Problema de batidas rápidas no motor principal,Ruído anormal nas escovas do motor durante a o...,Temperatura elevada no motor elétrico,Aquecimento excessivo do comutador em motores,Contatos elétricos que ficam grudados por solda,Contatos elétricos não duram muito tempo,...,Problema de barulho anormal na operação do mot...,Aquecimento excessivo nos rolamentos de bucha,Rolamento antifricção aquecendo além do normal,Controlador enfrentando problemas de sobrecarg...,Escovas produzindo faíscas excessivas,Escurecimento anormal do comutador em motores ...,Faíscas intermitentes afetando o desempenho da...,Acoplamento com amortecedores ou pinos de borr...,Problema de fumaça e solda na armadura,Motor não inicia
4,A bomba está piscando e emitindo calor excessivo.,Problema de eficiência energética na bomba,Zumbido perturbador proveniente do motor de co...,Motor elétrico emitindo cliques regulares,Batidas constantes e rápidas no motor elétrico,Escovas saltando ou tremendo no motor elétrico,Problema de superaquecimento no motor,Motor

In [37]:
import random

min_size = min([len(new_df[column].dropna()) for column in new_df.columns])
print(min_size)

def undersample_dataframe(df, min_size, random_seed=None):
    random.seed(random_seed)
    undersampled_dfs = []

    for column in df.columns:
        column_data = df[column].dropna().tolist()
        undersampled_column = random.sample(column_data, min_size)
        undersampled_dfs.append(undersampled_column)

    undersampled_df = pd.DataFrame(undersampled_dfs).T
    undersampled_df.columns = df.columns

    return undersampled_df

undersampled_df = undersample_dataframe(new_df, min_size)
undersampled_df.shape

328


(328, 116)

In [8]:
undersampled_df.to_excel(r'..\database\problems\undersampling_test.xlsx')

In [129]:
from sklearn.model_selection import train_test_split

def prepare_data(df, test_size=0.3, random_state=None):
    classes = df.columns.tolist()

    X = []
    y = []

    for _, row in df.iterrows():
        X.append(row.values)
        y.append(classes)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test


In [130]:
X_train, X_test, y_train, y_test = prepare_data(new_df)